In [1]:
import sys
sys.path.insert(0, '../')
from nn_cno import ode
import numpy as np
import itertools
import jax.numpy as jnp
import jax
import diffrax
import optax


In [10]:
c = ode.logicODE("../nn_cno/datasets/working_case_study/PKN-test.sif",
    "../nn_cno/datasets/working_case_study/MD-test.csv")
c.preprocessing()

In [3]:
sim  = c.simulate()

In [7]:
params = jnp.array(list(c.get_ODEparameters().values()))

In [11]:
%%timeit -n 1 -r 1
c.loss_function(params)

3.33 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [12]:
%%timeit 
c.loss_function(params)

189 µs ± 1.94 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [14]:
%%timeit -n 1 -r 1
c.loss_function_grad(params)

1min 1s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [15]:
%%timeit
c.loss_function_grad(params)

7.8 ms ± 88.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [19]:
c.fit(max_iter=10)

step 0, loss: 0.1511961966753006
	params: [1.0099999  2.01       0.09000007 1.         2.         0.10999993
 0.99000007 1.9900001  0.99000007 1.9900001  0.10999993 1.0099999
 2.0099998  1.         2.         0.09000007 1.         2.
 0.09000007 1.         2.         0.10999993]


DeviceArray([1.0872488 , 2.0917668 , 0.06310978, 1.        , 2.        ,
             0.18981409, 0.8983991 , 1.8984321 , 0.9014373 , 1.9042059 ,
             0.19754875, 1.0726762 , 2.0754545 , 1.0003778 , 2.        ,
             0.08042196, 1.        , 2.        , 0.00512886, 1.        ,
             2.        , 0.19432251], dtype=float32)

In [12]:
from functools import partial

In [12]:
lb = np.zeros(params.shape)
ub = np.ones(params.shape)
dz = 0.1*np.ones(params.shape)

def penalty_fn (x,lb,dz):
    dept = (x-lb)/dz
    p = 1e3*jnp.power(dept,2)
    return(p)



@jax.jit #@partial(jax.jit,static_argnums=(1,2,3))
def penalty(params):

    mse = jnp.where(params < lb+dz, 1, 0) * jnp.power((params-lb)/dz,2)*1e3
    mse += jnp.where(params > ub-dz, 1, 0) * jnp.power((ub-params)/dz,2)*1e3
    
    return mse.sum()


In [13]:
params = jnp.array(list(c.get_ODEparameters().values()))
penalty(params)

DeviceArray(800000., dtype=float32)

In [33]:
jax.value_and_grad(penalty)(params)

(DeviceArray(806000., dtype=float32),
 DeviceArray([     0., 200000.,  20000.,      0., 200000.,  20000.,
                   0., 200000.,      0., 200000.,  20000.,      0.,
              200000.,      0., 200000.,  20000.,      0., 200000.,
               20000.,      0., 200000.,  20000.], dtype=float32))

In [7]:
lb = c.ODEparameters_lb.copy()
ub = c.ODEparameters_ub.copy()
dz = c.ODEparameters_dz.copy()

@jax.jit
def this_loss(params):
    mse = 0.0
    for i in range(len(params)):
        if params[i] <= lb[i]+dz[i]:
            dept = (params[i]-lb[i])/dz[i]
            penalty = 1e3*jnp.power(dept,2)
            mse += penalty
            
        elif params[i] > ub[i]-dz[i]:
            dept = (ub[i] - params[i])/dz[i]
            penalty = 1e3*jnp.power(dept,2)
            mse += penalty

    #params = jnp.clip(params, lb, ub)
    return mse

In [38]:
print(lb.shape)
print(ub.shape)
print(dz.shape)
print(params.shape)

(22,)
(22,)
(22,)
(22,)


In [39]:
loss_value, grads = jax.value_and_grad(this_loss)(params)

ConcretizationTypeError: Abstract tracer value encountered where concrete value is expected: Traced<ShapedArray(bool[])>with<DynamicJaxprTrace(level=0/1)>
The problem arose with the `bool` function. 
While tracing the function this_loss at /var/folders/cx/9kyr3rt90c974wdygym_lhgh0000gn/T/ipykernel_97931/2333120683.py:5 for jit, this concrete value was not available in Python because it depends on the value of the argument 'params'.

See https://jax.readthedocs.io/en/latest/errors.html#jax.errors.ConcretizationTypeError

In [10]:
print(lb.shape)
print(ub.shape)
print(dz.shape)
print(params.shape)




(22,)
(22,)
(22,)
(22,)


DeviceArray([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
             0., 0., 0., 0., 0., 0., 0.], dtype=float32)